In [22]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

files = ['coretech.txt.zip', 'background.txt.zip', 'whycolab.txt.zip', 'backgroundAccept.txt.zip', 'backgroundDeny.txt.zip']

filename = files[4]


In [8]:
f = open('test.txt')
f.read()
# vocabulary = tf.compat.as_str(f).split()
print(f)
# print('Data size', len(f))


FileNotFoundError: [Errno 2] No such file or directory: 'test.txt'

In [23]:
# Read the data into a list of strings.
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words."""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data

vocabulary = read_data(filename)
print('Data size', len(vocabulary))

Data size 52720


In [24]:
# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 3000
randomWords = []



def build_dataset(words, n_words):
  """Process raw inputs into a dataset."""
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(n_words - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    word = word.strip(',')
    word = word.strip('.')
    word = word.strip('?')
    word = word.strip('!')
    word = word.strip('"')
    if word in dictionary:
      index = dictionary[word]
    else:
      randomWords.append(word)

      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  print(len(reversed_dictionary))
  return data, count, dictionary, reversed_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                            vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:50])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

print(randomWords)

3000
Most common words (+UNK) [['UNK', 5359], ('and', 2593), ('I', 1977), ('to', 1798), ('a', 1590), ('the', 1543), ('in', 1183), ('of', 1161), ('my', 730), ('with', 644), ('have', 635), ('design', 509), ('for', 455), ('as', 445), ('that', 444), ('am', 359), ('at', 332), ('on', 300), ('me', 299), ('an', 294), ('is', 291), ('experience', 276), ('from', 259), ('be', 243), ('skills', 232), ('can', 230), ('also', 223), ('background', 197), ('My', 193), ('team', 171), ('work', 170), ('this', 162), ('will', 158), ('working', 156), ('business', 153), ('how', 147), ('are', 131), ('worked', 126), ('new', 125), ('As', 124), ('about', 121), ('different', 117), ('product', 114), ('which', 111), ('bring', 111), ('believe', 111), ('thinking', 109), ('well', 108), ('think', 105), ('it', 103)]
Sample data [2, 10, 1260, 1, 816, 7, 0, 0, 13, 19] ['I', 'have', 'breadth', 'and', 'depth', 'of', 'UNK', 'UNK', 'as', 'an']
['expeirence', 'Professionally', 'Marketer', 'EIR', 'chatbot', 'months', 'generally', '

In [11]:

# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [skip_window]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])


10 have -> 1260 breadth
10 have -> 2 I
1260 breadth -> 1 and
1260 breadth -> 10 have
1 and -> 816 depth
1 and -> 1260 breadth
816 depth -> 7 of
816 depth -> 1 and


In [12]:

# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()


In [13]:

# Step 5: Begin training.
num_steps = 50001 # was 100001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print('Initialized')

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step ', step, ': ', average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = 'Nearest to %s:' % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = '%s %s,' % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()


Initialized
Average loss at step  0 :  165.808929443
Nearest to makeathon: (such, surrounding, forward., hackathons,, wireframing,, Here,, anticipate, centric,
Nearest to these: Management, MBA, Beyond, transmit, cycle, intense, structured, Makeathon,
Nearest to like: promote, vision, partner, studies, anthropology,, web, feedback., solver.,
Nearest to strong: Web, entrepreneur, Arts, tackling, Especially, therefore, HCI, record,
Nearest to been: objects,, potential, ", fabrication, health, understandings, capacity, technologist,,
Nearest to learning: looks, IT, difficult, effective, process,, Secondly,, roadmap., conferences,
Nearest to can: enthusiasm, analyst, areas., allow, area, capital,, tech,, person.,
Nearest to product: today, program., learner,, future,, know, contexts., feature, teammate,
Nearest to people: Things, entrepreneurial, becoming, i'm, viability,, outside, Working, Island,
Nearest to would: curious,, that, interior, light, Our, concepts., researching, digger.,
Nea

Average loss at step  42000 :  3.39045015037
Average loss at step  44000 :  3.36609467399
Average loss at step  46000 :  3.36575702369
Average loss at step  48000 :  3.35753019321
Average loss at step  50000 :  3.34222620034
Nearest to makeathon: Makeathon, relevancy, travelled, speak, bright, year., (such, project,
Nearest to these: intense, Beyond, complicated, everyone, Java, multiple, outside, transmit,
Nearest to like: promote, vision, smart, partner, use., lens, mentioned, facilitate,
Nearest to strong: Web, consideration, knowing, afforded, earning, diverse, short, therefore,
Nearest to been: objects,, internships,, York, worked, held, technically, machine, interested,
Nearest to learning: Blockchain, providing, whatever, looks, identify, processes, senior, problem,
Nearest to can: would, tech,, could, qualitative, enthusiasm, I’d, perfect, connect,
Nearest to product: landscape,, future,, factors, teamwork, learner,, contexts., motion, interviews,,
Nearest to people: Things, co

['UNK', 'and', 'I', 'to', 'a', 'the', 'in', 'of', 'my', 'with', 'have', 'design', 'for', 'as', 'that', 'am', 'at', 'on', 'me', 'an', 'is', 'experience', 'from', 'be', 'skills', 'can', 'also', 'background', 'My', 'team', 'work', 'this', 'will', 'working', 'business', 'how', 'are', 'worked', 'new', 'As', 'about', 'different', 'product', 'which', 'bring', 'believe', 'thinking', 'well', 'think', 'it', 'through', 'has', 'was', 'by', 'research', 'make', "I'm", 'into', 'people', 'design,', 'these', 'ideas', 'we', '"', 'but', 'been', 'valuable', 'projects', 'our', 'not', 'love', 'help', 'would', "I've", 'both', 'strong', 'more', 'technology', 'In', 'creative', 'makeathon', 'technical', 'able', 'or', 'create', '"I', 'like', 'user', 'I’ve', 'designer', 'teams', 'all', 'experiences', 'visual', 'learning', 'project', 'making', 'what', 'years', 'their', 'learned', 'This', 'diverse', 'because', 'where', 'use', 'designing', 'such', 'process', 'makeathon.', 'human', 'had', 'design.', 'most', 'who', 'u

In [20]:
# # Step 6: Visualize the embeddings.


def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
  assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

  plt.savefig(filename)

try:
  # pylint: disable=g-import-not-at-top
  from sklearn.manifold import TSNE
  # import matplotlib
  # matplotlib.use('TkAgg')
  import matplotlib.pyplot as plt
  %matplotlib inline

  tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
  plot_only = 250
  low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
  labels = [reverse_dictionary[i] for i in xrange(plot_only)]
  print(labels)

  plot_with_labels(low_dim_embs, labels)

except ImportError:
  print('Please install sklearn, matplotlib, and scipy to show embeddings.')